In [ ]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [1]:
import pandas as pd
from myUtils import pickle_load, symb_perf_stats_vectorized
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [2]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [ ]:
df = df_OHLCVA[-500::]
df

In [6]:
_df = df.xs('Volume', level=1, axis=1)

In [7]:
_df

,A,AA,AADI,AAIN,AAL,AAMC,AAN,AAON,AAP,AAPL,...,ZIP,ZLAB,ZM,ZNH,ZNTL,ZS,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-17,1571500.0,5443300,94220.0,NaN,61956500.0,NaN,NaN,226800.0,1073200.0,74271000.0,...,NaN,551300.0,6972100.0,28800.0,198000.0,1347400.0,2092100.0,1214300.0,154100.0,560800.0
2020-11-18,2047900.0,6255300,70740.0,NaN,99888500.0,NaN,NaN,282900.0,881600.0,76322100.0,...,NaN,415600.0,10974200.0,71300.0,185500.0,1272800.0,2439400.0,1457800.0,231800.0,666000.0
2020-11-19,1886400.0,5024600,134500.0,NaN,58809400.0,NaN,NaN,155300.0,1706700.0,74113000.0,...,NaN,543300.0,8007000.0,25900.0,169000.0,1474700.0,6567000.0,1329000.0,116600.0,560900.0
2020-11-20,2775400.0,4749800,59187.0,NaN,58685500.0,NaN,NaN,377900.0,1334400.0,73604300.0,...,NaN,184100.0,12586600.0,15600.0,330800.0,1491400.0,2837400.0,1217600.0,229500.0,577000.0
2020-11-23,3258700.0,7392400,47007.0,NaN,100764000.0,NaN,NaN,134200.0,1087700.0,127959300.0,...,NaN,240100.0,8486000.0,41700.0,248500.0,1511000.0,2600400.0,1493100.0,387700.0,693400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-04,1272100.0,11097600,118200.0,0.0,28222600.0,700.0,334900.0,106300.0,849600.0,140716700.0,...,728200.0,1520500.0,3968300.0,16300.0,355900.0,4320800.0,4377700.0,4314000.0,168000.0,1000200.0
2022-11-07,1213600.0,4790700,85400.0,500.0,23797300.0,1700.0,347800.0,107800.0,782300.0,83374600.0,...,622700.0,1819300.0,3317800.0,12400.0,381600.0,3390600.0,2504200.0,3940800.0,172900.0,1587400.0
2022-11-08,1028100.0,4740200,97500.0,100.0,25415700.0,700.0,432300.0,692800.0,571200.0,89908500.0,...,958400.0,692100.0,3987100.0,6100.0,392100.0,2239500.0,1846900.0,2552400.0,311800.0,2154600.0


In [37]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
import numpy as np
i, c = np.where(_df == 0)
print (f'i: {i}, c:{c}')
# print (next(iter(_df.columns[c]), 'no match'))

i: [  0   0   0 ... 498 498 498], c:[  35   97  642 ... 3522 3589 3594]


In [49]:
l_0 = list(set(c))
l_0.sort()
print(f'{len(l_0)}\n{l_0}')

559
[3, 5, 28, 29, 30, 31, 33, 35, 45, 51, 55, 59, 61, 62, 68, 71, 81, 82, 83, 85, 90, 91, 93, 97, 109, 111, 112, 117, 121, 128, 152, 153, 156, 166, 185, 216, 218, 225, 226, 234, 239, 242, 247, 251, 253, 255, 260, 275, 276, 285, 290, 291, 312, 316, 322, 338, 340, 344, 361, 377, 383, 396, 399, 404, 413, 419, 443, 451, 467, 480, 482, 486, 487, 496, 501, 506, 507, 510, 516, 520, 522, 523, 524, 527, 535, 537, 544, 558, 559, 562, 575, 577, 593, 595, 614, 618, 640, 642, 650, 658, 667, 668, 669, 670, 683, 690, 693, 707, 712, 717, 718, 721, 730, 733, 735, 743, 760, 761, 767, 775, 780, 791, 793, 803, 807, 824, 864, 875, 878, 892, 898, 905, 910, 913, 918, 928, 934, 942, 957, 963, 969, 970, 983, 1004, 1008, 1012, 1020, 1024, 1038, 1055, 1056, 1057, 1058, 1066, 1079, 1082, 1087, 1088, 1090, 1091, 1096, 1097, 1100, 1106, 1108, 1110, 1117, 1126, 1140, 1145, 1147, 1148, 1157, 1161, 1180, 1182, 1215, 1225, 1226, 1228, 1233, 1235, 1245, 1247, 1250, 1259, 1262, 1264, 1268, 1279, 1282, 1292, 1294, 1300, 

In [53]:
symbols_no_volume = _df.columns[l_0]
symbols_no_volume

Index(['AAIN', 'AAMC', 'ACAB', 'ACAD', 'ACAQ', 'ACBA', 'ACDI', 'ACEV', 'ACNT',
       'ADAL',
       ...
       'WLYB', 'WMPN', 'WNNR', 'WQGA', 'WRAC', 'WTMA', 'WWAC', 'XFIN', 'XPER',
       'YNDX'],
      dtype='object', length=559)